In [1]:
%%HTML
<style type='text/css'>
    *{
        # background-color:#E3EDCD;
        # background-color:black;
        # color:white;
        
    }
    h1{
        color:#1976d2;
    }
    h2{
        color:#f57c00;
    }
    h3{
        color:#ba37ff;
    }
    h4{
        color:green;
    }
    table{
        border:1px solid black !important;
        border-collapse:collapse !important;
    }
    th{
        background-color:blueviolet !important;
        text-align:center;
        color:white;
    }
    th,td{
        border:0.1px solid black !important;
        transition:0.2s all liner;
        
    }
    td:hover{
        transform:scale(1.1);
        background-color:orange;
        color:blueviolet;
    }
    .raw{
        white-space:pre;
    }
    .atcenter{
        text-align:center !important;
    }
    #imp{
        color:red;
        font-weight:bolder;
    }
</style>

# 天猫订单数据可视化优化版

## 导入需要的包

In [2]:
from pyecharts.globals import CurrentConfig,NotebookType
import pyecharts.options as opts
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB
from pyecharts.charts import Bar,Line,Pie,Map,Timeline
import pandas as pd
import numpy as np

本数据集共手机了发生在一个月内的28010条数据,包含  
- 订单编号 
- 总金额  
- 买家实际支付金额: 总金额-退款金额  
- 收货地址:各个省份
- 订单创建时间
- 订单付款时间
- 退款金额: 付款后申请退款的金额,如未付款,退款金额为0

## 数据准备阶段

In [3]:
data = pd.read_csv('../others/tmall_order_report.csv')

In [4]:
data.head()

,订单编号,总金额,买家实际支付金额,收货地址,订单创建时间,订单付款时间,退款金额
0,1,178.8,0.0,上海,2022-03-21 00:00:00,NaN,0.0
1,2,21.0,21.0,内蒙古自治区,2022-03-20 23:59:54,2022-03-21 00:00:02,0.0
2,3,37.0,0.0,安徽省,2022-03-20 23:59:35,NaN,0.0
3,4,157.0,157.0,湖南省,2022-03-20 23:58:34,2022-03-20 23:58:44,0.0
4,5,64.8,0.0,江苏省,2022-03-20 23:57:04,2022-03-20 23:57:11,64.8


In [5]:
data.tail()

,订单编号,总金额,买家实际支付金额,收货地址,订单创建时间,订单付款时间,退款金额
28005,28006,37.0,37.0,四川省,2022-03-27 00:01:00,2022-03-27 00:01:10,0.0
28006,28007,69.0,0.0,上海,2022-03-27 00:00:18,NaN,0.0
28007,28008,69.0,0.0,上海,2022-03-27 00:00:17,NaN,0.0
28008,28009,37.0,37.0,辽宁省,2022-03-27 00:00:09,2022-03-27 00:00:17,0.0
28009,28010,37.0,37.0,广东省,2022-03-27 00:00:06,2022-03-27 00:00:11,0.0


In [6]:
data.shape

(28010, 7)

In [7]:
data.columns

Index(['订单编号', '总金额', '买家实际支付金额', '收货地址 ', '订单创建时间', '订单付款时间 ', '退款金额'], dtype='object')

### 查看数据列中是否有空值

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28010 entries, 0 to 28009
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   订单编号      28010 non-null  int64  
 1   总金额       28010 non-null  float64
 2   买家实际支付金额  28010 non-null  float64
 3   收货地址      28010 non-null  object 
 4   订单创建时间    28010 non-null  object 
 5   订单付款时间    24087 non-null  object 
 6   退款金额      28010 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 1.5+ MB


#### ↑可以看出是有空值的,如索引5订单付款时间列非空的数据只有24087个,不足28010个

### 数据处理,去掉列明中的文字左右空格

In [9]:
data.columns

Index(['订单编号', '总金额', '买家实际支付金额', '收货地址 ', '订单创建时间', '订单付款时间 ', '退款金额'], dtype='object')

#### 可以看到收货地址和订单付款时间右边都有空格

In [10]:
data.columns.str.strip() #利用.strip()去除列名的左右空格

Index(['订单编号', '总金额', '买家实际支付金额', '收货地址', '订单创建时间', '订单付款时间', '退款金额'], dtype='object')

#### 只是这样写是不行的,因为并没有对原来的数据进行修改,我们需要对其进行修改

In [11]:
data.columns = data.columns.str.strip() #这样就可以了

### 查看是否有重复值,使用duplicated()方法,如果不重复就是False

In [12]:
data.duplicated() 

0        False
1        False
2        False
3        False
4        False
         ...  
28005    False
28006    False
28007    False
28008    False
28009    False
Length: 28010, dtype: bool

In [13]:
data.duplicated().sum() 
#将非空的行进行求和,如果说最后的数值是0则说明全都不重复

np.int64(0)

### 查看是否有缺失值

In [14]:
np.any(data.isnull())
#这个的含义是如果数据(data)有任何(any)空(isnull())就返回True

np.True_

#### 查看每一列的空值数量

In [15]:
data.isnull().sum()

订单编号           0
总金额            0
买家实际支付金额       0
收货地址           0
订单创建时间         0
订单付款时间      3923
退款金额           0
dtype: int64

#### ↑可以看到订单退款时间的缺失值还是很多的,有3923条,之后我们会对其进行处理

## 数据可视化

### 再看数据

In [16]:
data.head()

,订单编号,总金额,买家实际支付金额,收货地址,订单创建时间,订单付款时间,退款金额
0,1,178.8,0.0,上海,2022-03-21 00:00:00,NaN,0.0
1,2,21.0,21.0,内蒙古自治区,2022-03-20 23:59:54,2022-03-21 00:00:02,0.0
2,3,37.0,0.0,安徽省,2022-03-20 23:59:35,NaN,0.0
3,4,157.0,157.0,湖南省,2022-03-20 23:58:34,2022-03-20 23:58:44,0.0
4,5,64.8,0.0,江苏省,2022-03-20 23:57:04,2022-03-20 23:57:11,64.8


## 用地图展示各省份的订单量

#### 可以通过省份出现的次数得出每个省份的订单量是多少

<p id='imp'>简单理解:因为每个订单中都有收货地址的省份信息,我们可以<br>
    将其看作是该省份订单+1的操作,出现一个就加一个,这样就得到了每个省份对应的订单数量</p>

In [17]:
data['收货地址'].value_counts()

收货地址
上海          3353
广东省         2463
江苏省         2126
浙江省         2061
北京          2054
四川省         2019
山东省         1804
辽宁省         1187
天津          1153
湖南省         1099
河北省         1083
重庆          1036
河南省          966
云南省          778
安徽省          609
陕西省          536
福建省          489
山西省          465
广西壮族自治区      436
江西省          411
吉林省          401
黑龙江省         379
贵州省          345
内蒙古自治区       215
海南省          178
甘肃省          167
湖北省           75
新疆维吾尔自治区      58
宁夏回族自治区       42
青海省           19
西藏自治区          3
Name: count, dtype: int64

因为数据如上海,北京,天津这样的只写了名称没有跟上'市'这样的后缀, 
所以我们要使用一个函数来将这些市区处理一下

In [18]:
def province_map(p):
    if p in ['北京','上海','天津','重庆']:
        return p+'市'
    else:
        return p

In [19]:
data['收货地址']

0            上海
1        内蒙古自治区
2           安徽省
3           湖南省
4           江苏省
          ...  
28005       四川省
28006        上海
28007        上海
28008       辽宁省
28009       广东省
Name: 收货地址, Length: 28010, dtype: object

In [20]:
## 然后调用上面我们定义的函数并使用 数据.map(function)对数据进行整体操作
data['收货地址'] = data['收货地址'].map(province_map)

In [21]:
# 再次查看数据
data['收货地址'].value_counts()

收货地址
上海市         3353
广东省         2463
江苏省         2126
浙江省         2061
北京市         2054
四川省         2019
山东省         1804
辽宁省         1187
天津市         1153
湖南省         1099
河北省         1083
重庆市         1036
河南省          966
云南省          778
安徽省          609
陕西省          536
福建省          489
山西省          465
广西壮族自治区      436
江西省          411
吉林省          401
黑龙江省         379
贵州省          345
内蒙古自治区       215
海南省          178
甘肃省          167
湖北省           75
新疆维吾尔自治区      58
宁夏回族自治区       42
青海省           19
西藏自治区          3
Name: count, dtype: int64

#### 统计各个省份的订单量

In [22]:
data['订单付款时间'].notnull() #首先来看看订单付款时间不为空的数据

0        False
1         True
2        False
3         True
4         True
         ...  
28005     True
28006    False
28007    False
28008     True
28009     True
Name: 订单付款时间, Length: 28010, dtype: bool

In [23]:
data = data[data['订单付款时间'].notnull()] #利用掩码操作,只保留非空的数据
data

,订单编号,总金额,买家实际支付金额,收货地址,订单创建时间,订单付款时间,退款金额
1,2,21.0,21.0,内蒙古自治区,2022-03-20 23:59:54,2022-03-21 00:00:02,0.0
3,4,157.0,157.0,湖南省,2022-03-20 23:58:34,2022-03-20 23:58:44,0.0
4,5,64.8,0.0,江苏省,2022-03-20 23:57:04,2022-03-20 23:57:11,64.8
5,6,327.7,148.9,浙江省,2022-03-20 23:56:39,2022-03-20 23:56:53,178.8
6,7,357.0,357.0,天津市,2022-03-20 23:56:36,2022-03-20 23:56:40,0.0
...,...,...,...,...,...,...,...
28002,28003,77.0,77.0,重庆市,2022-03-27 00:02:39,2022-03-27 00:03:27,0.0
28003,28004,157.0,157.0,山东省,2022-03-27 00:01:42,2022-03-27 00:01:47,0.0
28005,28006,37.0,37.0,四川省,2022-03-27 00:01:00,2022-03-27 00:01:10,0.0
28008,28009,37.0,37.0,辽宁省,2022-03-27 00:00:09,2022-03-27 00:00:17,0.0


In [24]:
result = data.groupby('收货地址').count()
#进行分组,按照收货地址来分组
result

,订单编号,总金额,买家实际支付金额,订单创建时间,订单付款时间,退款金额
收货地址,,,,,,
上海市,3060,3060,3060,3060,3060,3060
云南省,667,667,667,667,667,667
内蒙古自治区,176,176,176,176,176,176
北京市,1853,1853,1853,1853,1853,1853
吉林省,336,336,336,336,336,336
四川省,1752,1752,1752,1752,1752,1752
天津市,1031,1031,1031,1031,1031,1031
宁夏回族自治区,40,40,40,40,40,40
安徽省,528,528,528,528,528,528


#### 我们可以看到上面数据表的所有行都变成了对应省份的订单数量,我们只选一个

In [25]:
result = result['订单编号']
result

收货地址
上海市         3060
云南省          667
内蒙古自治区       176
北京市         1853
吉林省          336
四川省         1752
天津市         1031
宁夏回族自治区       40
安徽省          528
山东省         1484
山西省          395
广东省         2022
广西壮族自治区      353
新疆维吾尔自治区      43
江苏省         1845
江西省          331
河北省          885
河南省          792
浙江省         1822
海南省          156
湖北省           57
湖南省          935
甘肃省          132
福建省          425
西藏自治区          2
贵州省          286
辽宁省         1012
重庆市          896
陕西省          441
青海省           18
黑龙江省         312
Name: 订单编号, dtype: int64

### 将其转换为字典形式

In [26]:
result.to_dict()

{'上海市': 3060,
 '云南省': 667,
 '内蒙古自治区': 176,
 '北京市': 1853,
 '吉林省': 336,
 '四川省': 1752,
 '天津市': 1031,
 '宁夏回族自治区': 40,
 '安徽省': 528,
 '山东省': 1484,
 '山西省': 395,
 '广东省': 2022,
 '广西壮族自治区': 353,
 '新疆维吾尔自治区': 43,
 '江苏省': 1845,
 '江西省': 331,
 '河北省': 885,
 '河南省': 792,
 '浙江省': 1822,
 '海南省': 156,
 '湖北省': 57,
 '湖南省': 935,
 '甘肃省': 132,
 '福建省': 425,
 '西藏自治区': 2,
 '贵州省': 286,
 '辽宁省': 1012,
 '重庆市': 896,
 '陕西省': 441,
 '青海省': 18,
 '黑龙江省': 312}

#### 再将其转化为列表包含元组的形式,这是我们画Map图所需要的数据格式

In [27]:
list(result.items())

[('上海市', 3060),
 ('云南省', 667),
 ('内蒙古自治区', 176),
 ('北京市', 1853),
 ('吉林省', 336),
 ('四川省', 1752),
 ('天津市', 1031),
 ('宁夏回族自治区', 40),
 ('安徽省', 528),
 ('山东省', 1484),
 ('山西省', 395),
 ('广东省', 2022),
 ('广西壮族自治区', 353),
 ('新疆维吾尔自治区', 43),
 ('江苏省', 1845),
 ('江西省', 331),
 ('河北省', 885),
 ('河南省', 792),
 ('浙江省', 1822),
 ('海南省', 156),
 ('湖北省', 57),
 ('湖南省', 935),
 ('甘肃省', 132),
 ('福建省', 425),
 ('西藏自治区', 2),
 ('贵州省', 286),
 ('辽宁省', 1012),
 ('重庆市', 896),
 ('陕西省', 441),
 ('青海省', 18),
 ('黑龙江省', 312)]

In [28]:
c = (
        Map()
        .add(
            '各省份订单量',
            list(result.items()),
            'china',
            is_map_symbol_show=False, #隐藏各个省份的符号
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(
                title='天猫订单各省份数量信息'
            ),
            visualmap_opts=opts.VisualMapOpts(
                range_text=['多','少'],
                min_=0,
                max_=3300,
                # range_color=['red','green','blue','purple']
            ),
            tooltip_opts=opts.TooltipOpts(
                background_color='orange',
                formatter='{a}<br>{b}:{c}个',
                textstyle_opts=opts.TextStyleOpts(
                    color='white'
                )
            )
        )
        .set_series_opts(
            label_opts=opts.LabelOpts(
                is_show = True
            )
        )
)

In [29]:
c.load_javascript()

In [30]:
c.render_notebook()

## 时间序列分析

In [31]:
data.head()

,订单编号,总金额,买家实际支付金额,收货地址,订单创建时间,订单付款时间,退款金额
1,2,21.0,21.0,内蒙古自治区,2022-03-20 23:59:54,2022-03-21 00:00:02,0.0
3,4,157.0,157.0,湖南省,2022-03-20 23:58:34,2022-03-20 23:58:44,0.0
4,5,64.8,0.0,江苏省,2022-03-20 23:57:04,2022-03-20 23:57:11,64.8
5,6,327.7,148.9,浙江省,2022-03-20 23:56:39,2022-03-20 23:56:53,178.8
6,7,357.0,357.0,天津市,2022-03-20 23:56:36,2022-03-20 23:56:40,0.0


In [32]:
data.info() #可以看到下面的订单创建时间和订单付款时间是Object类型

<class 'pandas.core.frame.DataFrame'>
Index: 24087 entries, 1 to 28009
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   订单编号      24087 non-null  int64  
 1   总金额       24087 non-null  float64
 2   买家实际支付金额  24087 non-null  float64
 3   收货地址      24087 non-null  object 
 4   订单创建时间    24087 non-null  object 
 5   订单付款时间    24087 non-null  object 
 6   退款金额      24087 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 1.5+ MB


#### 我们需要将订单创建时间和订单付款时间修改为时间类型

In [33]:
data['订单创建时间'] = pd.to_datetime(data['订单创建时间'])
data['订单付款时间'] = pd.to_datetime(data['订单付款时间'])

In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24087 entries, 1 to 28009
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   订单编号      24087 non-null  int64         
 1   总金额       24087 non-null  float64       
 2   买家实际支付金额  24087 non-null  float64       
 3   收货地址      24087 non-null  object        
 4   订单创建时间    24087 non-null  datetime64[ns]
 5   订单付款时间    24087 non-null  datetime64[ns]
 6   退款金额      24087 non-null  float64       
dtypes: datetime64[ns](2), float64(3), int64(1), object(1)
memory usage: 1.5+ MB


#### 可以看到修改成了datetime类

### 每天订单量的统计和可视化

<p id='imp'>
总体思路:<br>
既然是每天订单量的统计和可视化,那么就需要对数据进行按照时间分组,<br>
并统计相同日期的订单的出现次数并进行累计,这样就得到了每天的订单量
</p>

In [35]:
data['订单创建时间']

1       2022-03-20 23:59:54
3       2022-03-20 23:58:34
4       2022-03-20 23:57:04
5       2022-03-20 23:56:39
6       2022-03-20 23:56:36
                ...        
28002   2022-03-27 00:02:39
28003   2022-03-27 00:01:42
28005   2022-03-27 00:01:00
28008   2022-03-27 00:00:09
28009   2022-03-27 00:00:06
Name: 订单创建时间, Length: 24087, dtype: datetime64[ns]

In [36]:
# 得到订单创建时间
# 对数据进行整体操作,利用map(),并使用匿名函数将时间转换为字符串时间的格式
order_add_time = data['订单创建时间'].map(lambda x: x.strftime('%Y-%m-%d'))
# 如果对于strftime不熟悉就去看一下python高阶中的02python中的常见模块

In [37]:
order_add_time

1        2022-03-20
3        2022-03-20
4        2022-03-20
5        2022-03-20
6        2022-03-20
            ...    
28002    2022-03-27
28003    2022-03-27
28005    2022-03-27
28008    2022-03-27
28009    2022-03-27
Name: 订单创建时间, Length: 24087, dtype: object

In [38]:
# 以前的写法方便理解
result1 = data.groupby(order_add_time).count()
result1

,订单编号,总金额,买家实际支付金额,收货地址,订单创建时间,订单付款时间,退款金额
订单创建时间,,,,,,,
2022-03-01,163,163,163,163,163,163,163
2022-03-02,199,199,199,199,199,199,199
2022-03-03,239,239,239,239,239,239,239
2022-03-04,417,417,417,417,417,417,417
2022-03-05,333,333,333,333,333,333,333
2022-03-06,126,126,126,126,126,126,126
2022-03-07,156,156,156,156,156,156,156
2022-03-09,353,353,353,353,353,353,353
2022-03-10,23,23,23,23,23,23,23


In [39]:
# 我们只想要第一列
result1 = result1['订单编号']
result1

订单创建时间
2022-03-01     163
2022-03-02     199
2022-03-03     239
2022-03-04     417
2022-03-05     333
2022-03-06     126
2022-03-07     156
2022-03-09     353
2022-03-10      23
2022-03-11      15
2022-03-12       1
2022-03-13       5
2022-03-14       7
2022-03-15       4
2022-03-17     342
2022-03-18     838
2022-03-19     861
2022-03-20    1140
2022-03-21    1772
2022-03-22    1725
2022-03-23    1895
2022-03-24    1695
2022-03-25    2945
2022-03-26    2439
2022-03-27    2223
2022-03-28    2355
2022-03-29    1816
Name: 订单编号, dtype: int64

<p id='imp'>
    解惑:<br>
    为什么要进行转来转去的数据操作,看下面的result1的类型,它现在还是个series对象
</p>

In [40]:
print(type(result1)) #这时候它还只是个series对象

<class 'pandas.core.series.Series'>


In [41]:
# 将其转化为字典形式看看对不对得上号
result1 = result1.to_dict()
result1

{'2022-03-01': 163,
 '2022-03-02': 199,
 '2022-03-03': 239,
 '2022-03-04': 417,
 '2022-03-05': 333,
 '2022-03-06': 126,
 '2022-03-07': 156,
 '2022-03-09': 353,
 '2022-03-10': 23,
 '2022-03-11': 15,
 '2022-03-12': 1,
 '2022-03-13': 5,
 '2022-03-14': 7,
 '2022-03-15': 4,
 '2022-03-17': 342,
 '2022-03-18': 838,
 '2022-03-19': 861,
 '2022-03-20': 1140,
 '2022-03-21': 1772,
 '2022-03-22': 1725,
 '2022-03-23': 1895,
 '2022-03-24': 1695,
 '2022-03-25': 2945,
 '2022-03-26': 2439,
 '2022-03-27': 2223,
 '2022-03-28': 2355,
 '2022-03-29': 1816}

In [42]:
result1.keys()

dict_keys(['2022-03-01', '2022-03-02', '2022-03-03', '2022-03-04', '2022-03-05', '2022-03-06', '2022-03-07', '2022-03-09', '2022-03-10', '2022-03-11', '2022-03-12', '2022-03-13', '2022-03-14', '2022-03-15', '2022-03-17', '2022-03-18', '2022-03-19', '2022-03-20', '2022-03-21', '2022-03-22', '2022-03-23', '2022-03-24', '2022-03-25', '2022-03-26', '2022-03-27', '2022-03-28', '2022-03-29'])

In [43]:
c1 = (
    Line()
    .add_xaxis(list(result1.keys())) #让resul1t中各个数据的key作为横坐标,
    # 注意,传入的是字典的键,我们要将这些键组成一个列表,所以我们将其转化为列表了
    .add_yaxis('',list(result1.values()),areastyle_opts=opts.AreaStyleOpts(
        opacity=0.7
    ),is_smooth=True)
    #让result1中各个数据的values作为纵坐标
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title='天猫日销量统计'
        ),
        toolbox_opts=opts.ToolboxOpts(),
        tooltip_opts=opts.TooltipOpts(
             trigger='axis', #设置完触发方式后可视化效果会很好
        ),
        visualmap_opts=opts.VisualMapOpts(
            max_=3300,
            min_=1
        ),
        xaxis_opts=opts.AxisOpts(
            name='日期',
            name_location='end',
            name_rotate=30,
            boundary_gap=False
        ),
        yaxis_opts=opts.AxisOpts(
            name='订单量',
            name_location='end',
            # name_rotate=30
        )
        
    )
    .set_series_opts(
        markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(
                    type_='max',
                    itemstyle_opts=opts.ItemStyleOpts(
                        color='red'
                    )
                ),
                opts.MarkPointItem(
                    type_='min'
                ),
                
            ]
        ),
        markline_opts=opts.MarkLineOpts(
            label_opts=opts.LabelOpts(
                formatter='{b}:{c}'
            ),
            data=[
                opts.MarkLineItem(
                    type_='average',
                    name='均值线',
                )
            ]
        )
    )
)

In [44]:
c1.load_javascript()

In [45]:
c1.render_notebook()

## 每小时订单量统计可视化

#### 其实整体思路和刚才的是一样的

In [46]:
order_add_perhour = data['订单创建时间'].map(lambda x: x.strftime('%H'))

In [47]:
order_add_perhour

1        23
3        23
4        23
5        23
6        23
         ..
28002    00
28003    00
28005    00
28008    00
28009    00
Name: 订单创建时间, Length: 24087, dtype: object

可以看到我们获取到每个订单创建时间的小时数,这样进一步操作便可以得到哪个时间订单最多

In [48]:
# 按照时间进行分组,之后再根据出现的时间进行计数
result2 = data.groupby(order_add_perhour).count() 
result2

,订单编号,总金额,买家实际支付金额,收货地址,订单创建时间,订单付款时间,退款金额
订单创建时间,,,,,,,
00,880,880,880,880,880,880,880
01,449,449,449,449,449,449,449
02,289,289,289,289,289,289,289
03,163,163,163,163,163,163,163
04,113,113,113,113,113,113,113
05,96,96,96,96,96,96,96
06,210,210,210,210,210,210,210
07,476,476,476,476,476,476,476
08,744,744,744,744,744,744,744


In [49]:
# 还是只保留一个列
result2 = result2['订单编号']
result2

订单创建时间
00     880
01     449
02     289
03     163
04     113
05      96
06     210
07     476
08     744
09    1104
10    1444
11    1427
12    1179
13    1159
14    1280
15    1505
16    1297
17    1056
18    1099
19    1330
20    1686
21    1884
22    1809
23    1408
Name: 订单编号, dtype: int64

这样一来,我们便得到了每小时订单的数量

In [50]:
print(type(result2))

<class 'pandas.core.series.Series'>


可以看到上面的类型还是series类型

画一个柱状图,注意柱状图的数据格式是x,y轴都是列表类型,所以我们需要  
将上面的series类型先进行转换

In [51]:
result2 = result2.to_dict()
result2

{'00': 880,
 '01': 449,
 '02': 289,
 '03': 163,
 '04': 113,
 '05': 96,
 '06': 210,
 '07': 476,
 '08': 744,
 '09': 1104,
 '10': 1444,
 '11': 1427,
 '12': 1179,
 '13': 1159,
 '14': 1280,
 '15': 1505,
 '16': 1297,
 '17': 1056,
 '18': 1099,
 '19': 1330,
 '20': 1686,
 '21': 1884,
 '22': 1809,
 '23': 1408}

### 开始作图

In [84]:
c2 = (
    Bar()
    .add_xaxis(
        list(result2.keys()),
    )
    .add_yaxis(
        '',
        list(result2.values()),
    )
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(
            name = '时间',
            name_location='end',
            name_rotate=20,
        ),
        yaxis_opts=opts.AxisOpts(
            name='订单量',
            name_location='end',
        ),
        title_opts=opts.TitleOpts(
            title='每小时订单量'
        ),
        toolbox_opts=opts.ToolboxOpts(
            
        ),
        tooltip_opts=opts.TooltipOpts(
            trigger='axis',
            formatter='{b}时<br>{c}',
            background_color='orange',
            textstyle_opts=opts.TextStyleOpts(
                color='white'
            )
        ),
        visualmap_opts=opts.VisualMapOpts(
            min_=90,
            max_ = 2000,
        ),
        datazoom_opts=opts.DataZoomOpts()
    )
    .set_series_opts(
        markline_opts=opts.MarkLineOpts(
            label_opts=opts.LabelOpts(
                formatter='{b}:{c}'
            ),
            data=[
                opts.MarkLineItem(
                    type_='average',
                    name='平均值'
                )
            ]
        ),
        markpoint_opts=opts.MarkPointOpts(
            label_opts=opts.LabelOpts(
                formatter='{b}:{c}'
                    ),
            data=[
                opts.MarkPointItem(
                    
                    type_='max',
                    name='最大值',
                    itemstyle_opts=opts.ItemStyleOpts(
                        color='red'
                    )
                ),
                opts.MarkPointItem(
                    type_='min',
                    name='最小值'
                )
            ]
        )
    )
)

In [85]:
c2.load_javascript()

In [86]:
c2.render_notebook()